In [1]:
# look at tools/set_up_magics.ipynb
yandex_metrica_allowed = True ; get_ipython().run_cell('# one_liner_str\n\nget_ipython().run_cell_magic(\'javascript\', \'\', \'// setup cpp code highlighting\\nIPython.CodeCell.options_default.highlight_modes["text/x-c++src"] = {\\\'reg\\\':[/^%%cpp/]} ;\')\n\n# creating magics\nfrom IPython.core.magic import register_cell_magic, register_line_magic\nfrom IPython.display import display, Markdown, HTML\nimport argparse\nfrom subprocess import Popen, PIPE\nimport random\nimport sys\nimport os\nimport re\nimport signal\nimport shutil\nimport shlex\nimport glob\n\n@register_cell_magic\ndef save_file(args_str, cell, line_comment_start="#"):\n    parser = argparse.ArgumentParser()\n    parser.add_argument("fname")\n    parser.add_argument("--ejudge-style", action="store_true")\n    args = parser.parse_args(args_str.split())\n    \n    cell = cell if cell[-1] == \'\\n\' or args.no_eof_newline else cell + "\\n"\n    cmds = []\n    with open(args.fname, "w") as f:\n        f.write(line_comment_start + " %%cpp " + args_str + "\\n")\n        for line in cell.split("\\n"):\n            line_to_write = (line if not args.ejudge_style else line.rstrip()) + "\\n"\n            if line.startswith("%"):\n                run_prefix = "%run "\n                if line.startswith(run_prefix):\n                    cmds.append(line[len(run_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                run_prefix = "%# "\n                if line.startswith(run_prefix):\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                raise Exception("Unknown %%save_file subcommand: \'%s\'" % line)\n            else:\n                f.write(line_to_write)\n        f.write("" if not args.ejudge_style else line_comment_start + r" line without \\n")\n    for cmd in cmds:\n        display(Markdown("Run: `%s`" % cmd))\n        get_ipython().system(cmd)\n\n@register_cell_magic\ndef cpp(fname, cell):\n    save_file(fname, cell, "//")\n\n@register_cell_magic\ndef asm(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef makefile(fname, cell):\n    assert not fname\n    save_file("makefile", cell.replace(" " * 4, "\\t"))\n        \n@register_line_magic\ndef p(line):\n    try:\n        expr, comment = line.split(" #")\n        display(Markdown("`{} = {}`  # {}".format(expr.strip(), eval(expr), comment.strip())))\n    except:\n        display(Markdown("{} = {}".format(line, eval(line))))\n        \ndef show_file(file, clear_at_begin=True, return_html_string=False):\n    if clear_at_begin:\n        get_ipython().system("truncate --size 0 " + file)\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                elem.innerText = xmlhttp.responseText;\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (errors___OBJ__ < 10 && !entrance___OBJ__) {\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n        \n        <font color="white"> <tt>\n        <p id="__OBJ__" style="font-size: 16px; border:3px #333333 solid; background: #333333; border-radius: 10px; padding: 10px;  "></p>\n        </tt> </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__ -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n    \nBASH_POPEN_TMP_DIR = "./bash_popen_tmp"\n    \ndef bash_popen_terminate_all():\n    for p in globals().get("bash_popen_list", []):\n        print("Terminate pid=" + str(p.pid), file=sys.stderr)\n        p.terminate()\n    globals()["bash_popen_list"] = []\n    if os.path.exists(BASH_POPEN_TMP_DIR):\n        shutil.rmtree(BASH_POPEN_TMP_DIR)\n\nbash_popen_terminate_all()  \n\ndef bash_popen(cmd):\n    if not os.path.exists(BASH_POPEN_TMP_DIR):\n        os.mkdir(BASH_POPEN_TMP_DIR)\n    h = os.path.join(BASH_POPEN_TMP_DIR, str(random.randint(0, 1e18)))\n    stdout_file = h + ".out.html"\n    stderr_file = h + ".err.html"\n    run_log_file = h + ".fin.html"\n    \n    stdout = open(stdout_file, "wb")\n    stdout = open(stderr_file, "wb")\n    \n    html = """\n    <table width="100%">\n    <colgroup>\n       <col span="1" style="width: 70px;">\n       <col span="1">\n    </colgroup>    \n    <tbody>\n      <tr> <td><b>STDOUT</b></td> <td> {stdout} </td> </tr>\n      <tr> <td><b>STDERR</b></td> <td> {stderr} </td> </tr>\n      <tr> <td><b>RUN LOG</b></td> <td> {run_log} </td> </tr>\n    </tbody>\n    </table>\n    """.format(\n        stdout=show_file(stdout_file, return_html_string=True),\n        stderr=show_file(stderr_file, return_html_string=True),\n        run_log=show_file(run_log_file, return_html_string=True),\n    )\n    \n    cmd = """\n        bash -c {cmd} &\n        pid=$!\n        echo "Process started! pid=${{pid}}" > {run_log_file}\n        wait ${{pid}}\n        echo "Process finished! exit_code=$?" >> {run_log_file}\n    """.format(cmd=shlex.quote(cmd), run_log_file=run_log_file)\n    # print(cmd)\n    display(HTML(html))\n    \n    p = Popen(["bash", "-c", cmd], stdin=PIPE, stdout=stdout, stderr=stdout)\n    \n    bash_popen_list.append(p)\n    return p\n\n\n@register_line_magic\ndef bash_async(line):\n    bash_popen(line)\n    \n    \ndef show_log_file(file, return_html_string=False):\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function halt__OBJ__(elem, color)\n        {\n            elem.setAttribute("style", "font-size: 14px; background: " + color + "; padding: 10px; border: 3px; border-radius: 5px; color: white; ");                    \n        }\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            if (entrance___OBJ__ < 0) {\n                entrance___OBJ__ = 0;\n            }\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                if (elem.innerHTML != xmlhttp.responseText) {\n                                    elem.innerHTML = xmlhttp.responseText;\n                                }\n                                if (elem.innerHTML.includes("Process finished.")) {\n                                    halt__OBJ__(elem, "#333333");\n                                } else {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                }\n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (!entrance___OBJ__) {\n                                if (errors___OBJ__ < 6) {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                } else {\n                                    halt__OBJ__(elem, "#994444");\n                                }\n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n\n        <p id="__OBJ__" style="font-size: 14px; background: #000000; padding: 10px; border: 3px; border-radius: 5px; color: white; ">\n        </p>\n        \n        </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__.md -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n\n    \nclass TInteractiveLauncher:\n    tmp_path = "./interactive_launcher_tmp"\n    def __init__(self, cmd):\n        try:\n            os.mkdir(TInteractiveLauncher.tmp_path)\n        except:\n            pass\n        name = str(random.randint(0, 1e18))\n        self.inq_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".inq")\n        self.log_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".log")\n        \n        os.mkfifo(self.inq_path)\n        open(self.log_path, \'w\').close()\n        open(self.log_path + ".md", \'w\').close()\n\n        self.pid = os.fork()\n        if self.pid == -1:\n            print("Error")\n        if self.pid == 0:\n            exe_cands = glob.glob("../tools/launcher.py") + glob.glob("../../tools/launcher.py")\n            assert(len(exe_cands) == 1)\n            assert(os.execvp("python3", ["python3", exe_cands[0], "-l", self.log_path, "-i", self.inq_path, "-c", cmd]) == 0)\n        self.inq_f = open(self.inq_path, "w")\n        interactive_launcher_opened_set.add(self.pid)\n        show_log_file(self.log_path)\n\n    def write(self, s):\n        s = s.encode()\n        assert len(s) == os.write(self.inq_f.fileno(), s)\n        \n    def get_pid(self):\n        n = 100\n        for i in range(n):\n            try:\n                return int(re.findall(r"PID = (\\d+)", open(self.log_path).readline())[0])\n            except:\n                if i + 1 == n:\n                    raise\n                time.sleep(0.1)\n        \n    def input_queue_path(self):\n        return self.inq_path\n        \n    def close(self):\n        self.inq_f.close()\n        os.waitpid(self.pid, 0)\n        os.remove(self.inq_path)\n        # os.remove(self.log_path)\n        self.inq_path = None\n        self.log_path = None \n        interactive_launcher_opened_set.remove(self.pid)\n        self.pid = None\n        \n    @staticmethod\n    def terminate_all():\n        if "interactive_launcher_opened_set" not in globals():\n            globals()["interactive_launcher_opened_set"] = set()\n        global interactive_launcher_opened_set\n        for pid in interactive_launcher_opened_set:\n            print("Terminate pid=" + str(pid), file=sys.stderr)\n            os.kill(pid, signal.SIGKILL)\n            os.waitpid(pid, 0)\n        interactive_launcher_opened_set = set()\n        if os.path.exists(TInteractiveLauncher.tmp_path):\n            shutil.rmtree(TInteractiveLauncher.tmp_path)\n    \nTInteractiveLauncher.terminate_all()\n   \nyandex_metrica_allowed = bool(globals().get("yandex_metrica_allowed", False))\nif yandex_metrica_allowed:\n    display(HTML(\'\'\'<!-- YANDEX_METRICA_BEGIN -->\n    <script type="text/javascript" >\n       (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};\n       m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})\n       (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");\n\n       ym(59260609, "init", {\n            clickmap:true,\n            trackLinks:true,\n            accurateTrackBounce:true\n       });\n    </script>\n    <noscript><div><img src="https://mc.yandex.ru/watch/59260609" style="position:absolute; left:-9999px;" alt="" /></div></noscript>\n    <!-- YANDEX_METRICA_END -->\'\'\'))\n\ndef make_oneliner():\n    html_text = \'("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "")\'\n    html_text += \' + "<""!-- MAGICS_SETUP_PRINTING_END -->"\'\n    return \'\'.join([\n        \'# look at tools/set_up_magics.ipynb\\n\',\n        \'yandex_metrica_allowed = True ; get_ipython().run_cell(%s);\' % repr(one_liner_str),\n        \'display(HTML(%s))\' % html_text,\n        \' #\'\'MAGICS_SETUP_END\'\n    ])\n       \n\n');display(HTML(("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "") + "<""!-- MAGICS_SETUP_PRINTING_END -->")) #MAGICS_SETUP_END

<IPython.core.display.Javascript object>

# SSE

Streaming SIMD extensions

Single instruction multiple data

Сравним sse и x87 на 32-битной и 64-битной архитектуре

In [2]:
%%cpp double_mul.c
%run gcc -m32 -mfpmath=387 -masm=intel -O3 double_mul.c -S -o double_mul.S # x87, 32bit
%run cat double_mul.S | grep -v "^\s*\."
    
double mul(double a) { 
    return a * 0;
}

double mul2(double a, double b) { 
    return a * b;
}

Run: `gcc -m32 -mfpmath=387 -masm=intel -O3 double_mul.c -S -o double_mul.S # x87, 32bit`

Run: `cat double_mul.S | grep -v "^\s*\."`

mul:
	fldz
	fmul	QWORD PTR [esp+4]
	ret
mul2:
	fld	QWORD PTR [esp+12]
	fmul	QWORD PTR [esp+4]
	ret


In [3]:
%%cpp double_mul.c
%run gcc -m64 -mfpmath=387 -masm=intel -O3 double_mul.c -S -o double_mul.S # x87, 64bit
%run cat double_mul.S | grep -v "^\s*\."
    
double mul(double a) { 
    return a * 0;
}

double mul2(double a, double b) { 
    return a * b;
}

Run: `gcc -m64 -mfpmath=387 -masm=intel -O3 double_mul.c -S -o double_mul.S # x87, 64bit`

Run: `cat double_mul.S | grep -v "^\s*\."`

mul:
	movsd	QWORD PTR [rsp-8], xmm0
	fld	QWORD PTR [rsp-8]
	fmul	DWORD PTR .LC0[rip]
	fstp	QWORD PTR [rsp-8]
	movsd	xmm0, QWORD PTR [rsp-8]
	ret
mul2:
	movsd	QWORD PTR [rsp-8], xmm0
	fld	QWORD PTR [rsp-8]
	movsd	QWORD PTR [rsp-8], xmm1
	fld	QWORD PTR [rsp-8]
	fmulp	st(1), st
	fstp	QWORD PTR [rsp-8]
	movsd	xmm0, QWORD PTR [rsp-8]
	ret


In [4]:
%%cpp double_mul.c
%run gcc -m32 -mfpmath=sse -msse4 -masm=intel -O3 double_mul.c -S -o double_mul.S # SSE, 32bit (add -msse4!)
%run cat double_mul.S | grep -v "^\s*\."
    
double mul(double a) { 
    return a * 13;
}

double mul2(double a, double b) { 
    return a * b;
}

Run: `gcc -m32 -mfpmath=sse -msse4 -masm=intel -O3 double_mul.c -S -o double_mul.S # SSE, 32bit (add -msse4!)`

Run: `cat double_mul.S | grep -v "^\s*\."`

mul:
	sub	esp, 12
	movsd	xmm0, QWORD PTR .LC0
	mulsd	xmm0, QWORD PTR [esp+16]
	movsd	QWORD PTR [esp], xmm0
	fld	QWORD PTR [esp]
	add	esp, 12
	ret
mul2:
	sub	esp, 12
	movsd	xmm0, QWORD PTR [esp+24]
	mulsd	xmm0, QWORD PTR [esp+16]
	movsd	QWORD PTR [esp], xmm0
	fld	QWORD PTR [esp]
	add	esp, 12
	ret


In [5]:
%%cpp double_mul.c
%run gcc -m64 -mfpmath=sse -masm=intel -O3 double_mul.c -S -o double_mul.S # SSE, 64bit
%run cat double_mul.S | grep -v "^\s*\."
    
double mul(double a) { 
    return a * 0;
}

double mul2(double a, double b) { 
    return a * b;
}

Run: `gcc -m64 -mfpmath=sse -masm=intel -O3 double_mul.c -S -o double_mul.S # SSE, 64bit`

Run: `cat double_mul.S | grep -v "^\s*\."`

mul:
	mulsd	xmm0, QWORD PTR .LC0[rip]
	ret
mul2:
	mulsd	xmm0, xmm1
	ret


## Через боль и страдания пишем аналогичный ассемблерный код для SSE 32bit

In [6]:
%%cpp check_mul.c
%run gcc -msse4 -g3 -m32 -masm=intel check_mul.c -o check_mul.exe
%run ./check_mul.exe

#include <stdio.h>
#include <assert.h>

double mul(double a);
double mul2(double a, double b);
    
__asm__ (R"(
.text
mul:
    movsd    xmm0, [esp+4]  
    lea      eax, .mconst13
    mulsd    xmm0, QWORD PTR [eax]
    movsd    [esp+4], xmm0
    fld      QWORD PTR [esp+4]
    ret
mul2:
    movsd   xmm0, [esp+12]  
    mulsd   xmm0, QWORD PTR [esp+4]
    movsd   [esp+12], xmm0
    fld     QWORD PTR [esp+12]
    ret
.mconst13:
    .long 0
    .long 1076494336
)");

int main() {
    printf("mul(1.5) = %0.9lf\n", mul(1.5));
    printf("mul2(2.1, 20) = %0.9lf\n", mul2(2.1, 20));
    return 0;
}




Run: `gcc -msse4 -g3 -m32 -masm=intel check_mul.c -o check_mul.exe`

Run: `./check_mul.exe`

mul(1.5) = 19.500000000
mul2(2.1, 20) = 42.000000000


# Intrinsics

In [7]:
%%cpp bitmask.c
%run gcc -m32 -msse4 -O3 bitmask.c -S -o bitmask.S # SSE, 64bit
%run cat bitmask.S | grep -v "^\s*\."
  
    
#include <xmmintrin.h>
    
void bit_and(const int* __restrict__ a, 
             const int* __restrict__ b, 
             int* __restrict__ c) {
    for (int i = 0; i < 4 * 10; ++i) {
        c[i] = a[i] & b[i];
    }
}

void bit_and_2(const int* __restrict__ a, 
               const int* __restrict__ b, 
               int* __restrict__ c) {
    for (int i = 0; i < 10; i += 1) {
        ((__m128*)c)[i] = _mm_and_ps(((__m128*)a)[i], ((__m128*)b)[i]);
    }
}
    

Run: `gcc -m32 -msse4 -O3 bitmask.c -S -o bitmask.S # SSE, 64bit`

Run: `cat bitmask.S | grep -v "^\s*\."`

bit_and:
	pushl	%ebp
	pushl	%edi
	pushl	%esi
	pushl	%ebx
	subl	$28, %esp
	movl	48(%esp), %edx
	movl	52(%esp), %edi
	movl	56(%esp), %esi
	movl	%edx, %eax
	andl	$15, %eax
	shrl	$2, %eax
	negl	%eax
	andl	$3, %eax
	je	.L7
	movl	(%edx), %ecx
	andl	(%edi), %ecx
	cmpl	$1, %eax
	movl	%ecx, (%esi)
	je	.L8
	movl	4(%edx), %ecx
	andl	4(%edi), %ecx
	cmpl	$3, %eax
	movl	%ecx, 4(%esi)
	jne	.L9
	movl	8(%edi), %ecx
	andl	8(%edx), %ecx
	movl	$37, 4(%esp)
	movl	$3, (%esp)
	movl	%ecx, 8(%esi)
	movl	$40, %ecx
	movl	$36, %ebp
	movl	$9, 12(%esp)
	subl	%eax, %ecx
	movl	%ecx, 8(%esp)
	sall	$2, %eax
	leal	(%edi,%eax), %ebx
	leal	(%edx,%eax), %ecx
	addl	%esi, %eax
	cmpl	$10, 12(%esp)
	movdqu	(%ebx), %xmm0
	pand	(%ecx), %xmm0
	movups	%xmm0, (%eax)
	movdqu	16(%ebx), %xmm0
	pand	16(%ecx), %xmm0
	movups	%xmm0, 16(%eax)
	movdqu	32(%ebx), %xmm0
	pand	32(%ecx), %xmm0
	movups	%xmm0, 32(%eax)
	movdqu	48(%ebx), %xmm0
	pand	48(%ecx), %xmm0
	movups	%xmm0, 48(%eax)
	movdqu	64(%ebx), %xmm0
	pand	64(%ecx), %xmm0
	movups	%xmm0,

In [8]:
%%cpp bitmask_test.c
%run gcc -m64 -msse4 -O3 bitmask_test.c bitmask.c -o bitmask_test.exe # SSE, 64bit
%run ./bitmask_test.exe
 
#include <stdio.h>
#include <assert.h>
#include <xmmintrin.h>
    
void bit_and(const int* __restrict__ a, 
             const int* __restrict__ b, 
             int* __restrict__ c);
void bit_and_2(const int* __restrict__ a, 
               const int* __restrict__ b, 
               int* __restrict__ c);
    

int main() {
    char __attribute__((aligned(128))) ac[128 * 10 + 3] = "ahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjc";
    char __attribute__((aligned(128))) bc[128 * 10 + 3] = "ahjlascscsdafbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjcahjlvbshrvkbvjknkjsnbjskndsenckjwncskjncvsjckjsncksjncskjdnckjsncjksdncndkcnsdkjcnsdcjksndcjksdncjksdnjkdcnjknckdjc";
    char __attribute__((aligned(128))) c1c[128 * 10 + 3];
    char __attribute__((aligned(128))) c2c[128 * 10 + 3];
    bit_and((int*)ac, (int*)bc, (int*)c1c);
    bit_and_2((int*)ac, (int*)bc, (int*)c2c);
    printf("%p %p %p %p\n", ac, bc, c1c, c2c);
    for (int i = 0; i < 4 * 4 * 10; ++i) {
        assert(c1c[i] == c2c[i]);
    }
    return 0;
}

Run: `gcc -m64 -msse4 -O3 bitmask_test.c bitmask.c -o bitmask_test.exe # SSE, 64bit`

Run: `./bitmask_test.exe`

0x7ffe76e7e380 0x7ffe76e7e900 0x7ffe76e7ee80 0x7ffe76e7f400


## Полезные интринсики для векторных операций с float'ами

`_mm_dp_pd`, `_mm_dp_ps`, `_mm_add_epi8`, `_mm_loadu_ps`, `_mm_setzero_ps`, `_mm_mul_ss`, `_mm_add_ps`, `_mm_hadd_ps`, `_mm_cvtss_f32`